In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly
plotly.offline.init_notebook_mode()
from toolkit.tire_model.tire_model_pacejka_2010 import *
from toolkit.tire_model.tire_model_fitting import *
from toolkit.tire_model.ttc_loader import *
from toolkit.tire_model.tire_fitting_masks import *
from toolkit.tire_model.tire_model_utils import *
from toolkit.common.constants import *

In [ ]:
# Get the ttc data for the tire you want to fit
combi_runs, cornering, drive_brake, name = load_runs(get_R20_18x6_7_runs_raw())
# create all the boring lists and stuff
params_list: List = []
error_list: List = []

In [ ]:
# Let's just take a leap of fath and ignore this for a second, we'll come back to it
tire_model = tire_model_from_arr(H_R20_18X6_7)
fz_nom = tire_model.FNOMIN
pres_nom = tire_model.NOMPRES
# PHY1, PHY2, PVY1, PVY2 = 0.0, 0.0, 0.0, 0.0

In [ ]:
def plot_fit(fz_targ, func, sa, ia, pres, kappa, figs, dg, name, fz_err=False):
    dd = dg[np.abs(dg.FZ - fz_targ) < fz_targ*0.1+20]
    dd = dd[np.abs(dd.IA - ia) < np.deg2rad(0.5)]
    dd = filter_press(dd, press=pres, d_press=6000)
    fy = func(fz_targ, sa, ia, pres, kappa, offsets=True)
    fy_real = func(fz_targ, dd.SA, dd.IA, dd.P, dd.SL)
    shy, svy = None, None
    if type(fy) is tuple:
        shy = fy[1]
        svy = fy[2]
        fy = fy[0]
    error = np.mean(np.abs(fy_real - dd.FY))
    figs.add_trace(go.Scattergl(x=np.rad2deg(sa), y=fy, name=f"{name}\t{error:.2f} N rmse", legendgroup=f"{name}"))
    if fz_err:
        fz_err = (dd.FZ - fz_targ)
        figs.add_trace(go.Scattergl(x=np.rad2deg(dd.SA), y=dd.FZ, mode='markers', marker=dict(size=1.5, color=fz_err, colorscale='Viridis', showscale=True), legendgroup=f"{name}", showlegend=False))
    else:
        figs.add_trace(go.Scattergl(x=np.rad2deg(dd.SA), y=dd.FY, mode='markers', marker=dict(size=1), legendgroup=f"{name}", showlegend=False))
    if shy is not None:
        figs.add_trace(go.Scattergl(x=shy, y=svy, mode='markers', marker=dict(size=4, color='red'), legendgroup=f"{name}", showlegend=False))
    return error
slip_angles = np.linspace(-0.25, 0.25, 100)
sweeps = [220, 440, 660, 880, 1100]

In [ ]:
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[df.TSTC > 50] # only use the runs where the tire is warm, greater than 50 deg C
def model(fz_targ, sa, ia_targ, press, kappa, offsets=False):
    df_z = (fz_targ - fz_nom) / fz_nom
    mu_y = (tire_model.PDY1 + tire_model.PDY2 * df_z) * (1 - tire_model.PDY3 * ia_targ**2)
    kya = tire_model.PKY1 * fz_nom * np.sin(tire_model.PKY4*np.arctan(fz_targ/((tire_model.PKY2 + tire_model.PKY5*(ia_targ**2))*fz_nom))) * (1 - tire_model.PKY3 * np.abs(ia_targ))
    c_y = tire_model.PCY1 # now we get a coefficient for c_y
    d_y = mu_y * fz_targ
    e_y = (tire_model.PEY1 + tire_model.PEY2 * df_z) * (1 + (tire_model.PEY5 * ia_targ**2) - (tire_model.PEY3 + tire_model.PEY4 * ia_targ) * np.sign(sa))# now we get a coefficient for e_y
    b_y = kya / (d_y * c_y) # and then we can get b_y
    shy_0 = tire_model.PHY1 + tire_model.PHY2 * df_z
    svy_0 = (tire_model.PVY1 + tire_model.PVY2 * df_z) * fz_targ
    svy_g = (tire_model.PVY3 + tire_model.PVY4 * df_z) * fz_targ * ia_targ
    svy = svy_0 + svy_g
    kyg = (tire_model.PKY6 + tire_model.PKY7 * df_z) * fz_targ
    shy_g = (kyg * ia_targ - svy_g) / kya
    shy = shy_0 + shy_g
    sa_y = sa + shy
    fy = d_y * np.sin(c_y * np.arctan(b_y * sa_y - e_y * (b_y * sa_y - np.arctan(b_y * sa_y)))) + svy
    if offsets:
        return (fy, np.ones(1) * np.rad2deg(-shy), np.ones(1) * svy)
    return fy
fig = go.Figure()
i, err = 0.0, 0.0
for fza in sweeps:
    for pres in [8*PSI_TO_PA, 10*PSI_TO_PA, 12*PSI_TO_PA, 14*PSI_TO_PA]:
        error = plot_fit(fza, model, slip_angles, np.deg2rad(0.0), pres, 0.0, fig, df, f"Fit {fza} N {pres/1000:.1f} kPa")
        err += error
        i += 1
print(f"Total err: {(err/i):.2f}")
fig.update_layout(template="plotly_dark", title=f"Fy vs SA", xaxis_title="SA (deg)", yaxis_title="Fy (N)")
fig.update_yaxes(range=[-3000, 3000])
fig.show()

While looking at the runs for different pressures I found 
Something I noticed is that the 55kPa run at 1100N seems very strange, I am inclined to think that that the tire is starting to debead or leak air and then was subsequently filled<br>
But enough speculation lets just check

In [ ]:
df = filter_vel(cornering, 11.1, 0.1)
df = df[np.abs(df.IA) < np.deg2rad(0.5)]
df = remove_time_gaps(df)
fig = make_subplots(4, 1, shared_xaxes=True)
fig.add_scattergl(x=df.ET, y=df.P, row=1, col=1, mode="markers", marker=dict(size=1), name="Press")
fig.add_scattergl(x=df.ET, y=df.FY, row=2, col=1, mode="markers", marker=dict(size=1), name="FY")
fig.add_scattergl(x=df.ET, y=df.FZ, row=2, col=1, mode="markers", marker=dict(size=1), name="FZ")
fig.add_scattergl(x=df.ET, y=df.RL, row=3, col=1, mode="markers", marker=dict(size=1), name="RL")
fig.add_scattergl(x=df.ET, y=df.RE, row=3, col=1, mode="markers", marker=dict(size=1), name="RE")
fig.add_scattergl(x=df.ET, y=np.rad2deg(df.IA), row=4, col=1, mode="markers", marker=dict(size=1), name="IA")
fig.add_scattergl(x=df.ET, y=np.rad2deg(df.SA), row=4, col=1, mode="markers", marker=dict(size=1), name="SA")
fig.update_layout(template="plotly_dark", title=f"TTC run", xaxis_title="ET (sec)")
fig.show()

So that answers our question, at 55kPa and 1100N (and also slightly on the 880N) of $F_z$ the machine bottoms out with the spindle 0.2m off of the belt (look at RL)<br>
It is interesting to see how the machine has reasonably large spikes in $F_z$ as the tire returns within the operating range of the machine because the control loop has no way of controlling quickly enough because (assuming its using PID) the D term would be nulled our when the machine hits its travel limit<br>
<br>
Also It seems that as the machine can go below this when there is a inclination angle